In [1]:
import json
import boto3
from ExchangeRateEvent import *
from get_currencies import *
from app_database import *
from sqs import *
from constants import API_URL, BASE_CURRENCY, LOCAL_CURRENCIES
import logging
import os
from datetime import timedelta

In [2]:
logger.setLevel(logging.DEBUG)
logger = logging.getLogger()
log_handler = logging.FileHandler(filename='log.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
log_handler.setFormatter(formatter)
logger.addHandler(log_handler)

In [3]:
DB_URL = os.environ['DB_URL']
API_KEY = os.environ['API_KEY']

In [4]:
engine = connect_db_engine(DB_URL)

In [5]:
vw_latest_exchange_rate = latest_exchange_rate(engine, 'vw_latest_exchange_rate')

In [6]:
next_date = (vw_latest_exchange_rate[0][0] + timedelta(days = 1)).strftime('%Y-%m-%d')

In [8]:
test = get_currencies(
                      BASE_CURRENCY
                      ,LOCAL_CURRENCIES
                      ,next_date
                      ,API_KEY
                      ,API_URL
)

In [9]:
sql_values = [t.to_sql_values() for t in test]

In [10]:
sqs_messages = [t.to_sqs_message() for t in test]

In [11]:
insert_exchange_rate(engine, 'exchange_rates', sql_values)